## Cross Sections ##

This section is testing for the cross section script which looks at wind profiles under 4K warming, 4K cooling, and normal conditons. It also looks at warming and heating "anomolies" which are the deviation of these scenarios from the standard scenario. The script for HPCC use is called month_cross_sections.py and the results of the HPCC run can be found in figures.

In [ ]:
# libraries

import seaborn as sns
import xarray as xr
import numpy as np
import pandas as pd
import intake
import dask
import regionmask
import warnings
from pathlib import Path
from tqdm import tqdm
from xmip.preprocessing import combined_preprocessing, replace_x_y_nominal_lat_lon
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# data

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
    col = intake.open_esm_datastore(url)

In [ ]:
# load some monthly data from the GFDL-CM4 4-K warming experiment

query = dict(experiment_id =['amip-p4K'],
             variable_id=['ua', 'va'],
             source_id=['GFDL-CM4'],
             table_id='Amon'
            )

cat = col.search(**query)
print(cat.df['source_id'].unique())

# load data into dictionary
# keys are constructed as 'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

z_kwargs = {'consolidated': True, 'decode_times':True}

with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    dset_dict = cat.to_dataset_dict(zarr_kwargs=z_kwargs, preprocess=combined_preprocessing)


In [ ]:
print(dset_dict.keys())

In [ ]:
dsh = dset_dict['CMIP.NOAA-GFDL.GFDL-CM4.amip.Amon.gr1']
dsw = dset_dict['CFMIP.NOAA-GFDL.GFDL-CM4.amip-p4K.Amon.gr1']
dsc = dset_dict['CFMIP.NOAA-GFDL.GFDL-CM4.amip-m4K.Amon.gr1']

In [ ]:
data = dsh.sel(time=dsh.time.dt.month == 6)
data

In [ ]:
# get a couple datasets out of the dictionary

ds = dset_dict['CFMIP.NOAA-GFDL.GFDL-CM4.amip-p4K.Amon.gr1']

In [ ]:
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
          'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

monthly_data = {}
for i, mon in enumerate(months, start=1):
    data = ds.sel(time=ds.time.dt.month == i)
    monthly_data[mon] = data.mean(dim = 'time')

In [ ]:
all_data = ds.va.sel(x=slice(280, 320), y=-20.5)
global_min = all_data.min() # compute global min and max
global_max = all_data.max()

sns.axes_style('dark')
fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(12, 8), sharey=True)
axes = axes.flatten()

for ax, mon in zip(axes, months):
    im = monthly_data[mon].va.sel(x=slice(280, 320), y=-20.5).assign_coords(plev=ds['plev'] / 100).plot(ax=ax, add_colorbar=False)
    ax.set_ylim(1000, 400)
    ax.set_title(f'{mon} Meridional Wind')
    ax.set_xlabel('Longitude (°E)')
    ax.set_ylabel('Pressure (hPa)')

cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])  # [left, bottom, width, height]
fig.colorbar(im, cax=cbar_ax, label='Average Meridional Wind Speed (m/s)')  # Adjust label as needed

plt.tight_layout(rect=[0, 0, 0.9, 1])  # Leave space for colorbar and title
plt.savefig('../figures/MonthAverages.png', dpi=300)
plt.show()

## Retrieving Era 5 Data ##

This second section is for testing of the low level jet detection algorithm used in this paper. It should differentiate between coupled and uncoupled jets.

In [ ]:
# libraries

import seaborn as sns
import xarray as xr
import numpy as np
import pandas as pd
import dask
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
import cdsapi

client = cdsapi.Client()

In [ ]:
# example ERA5 retrieval script

dataset = 'reanalysis-era5-pressure-levels'

request = {
     'product_type': ['reanalysis'],
     'variable': ['geopotential'],
     'year': ['2024'],
     'month': ['03'],
     'day': ['01'],
     'time': ['13:00'],
     'pressure_level': ['1000'],
     'data_format': 'netcdf',      # Supported format: grib and netcdf. Default: grib
     'area': [60, -10, 50, 2],   # North, West, South, East. Default: global
     'grid': [1.0, 1.0],         # Latitude/longitude grid. Default: 0.25 x 0.25 
}

target = 'download.grib'  # Output file. Adapt as you wish.

client.retrieve(dataset, request, target) # this downloads it

or using Amazon Web Services (AWS)

In [ ]:
data = xr.open_dataset('../data/e5.oper.an.pl.128_132_v.ll025uv.2024080100_2024080123.nc', chunks='auto')
data

In [ ]:
mask = xr.open_dataset('../data/e5.oper.an.pl.128_129_z.ll025sc.2024080100_2024080123.nc', chunks='auto')
mask

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 8), sharey=True)
data.sel(longitude=slice(280, 320), latitude=-20.5).isel(time=10).V.plot(ax=ax)
ax.set_ylim(1000, 400)
ax.set_title(f'Meridional Wind')
ax.set_xlabel('Longitude (°E)')
ax.set_ylabel('Pressure (hPa)')


In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 8), sharey=True)
mask.sel(longitude=slice(280, 320), latitude=-20.5).isel(time=10).Z.plot(ax=ax)
ax.set_ylim(1000, 400)

## 6–Hourly Model Data ##

In [ ]:
# this is daily data with precipitation

query = dict(experiment_id =['amip', 'amip-p4K'],
             table_id='Eday',
             variable_id=['ua', 'va', 'prw', 'ta', 'ts', 'wap'],
             source_id='GFDL-CM4'
)

cat = col.search(**query)


In [ ]:
# load 6-hourly data with meridional and zonal wind, air temperature, surface pressure, and spefific humidity

query = dict(source_id=['GFDL-CM4'],# , 'MRI-ESM2-0', 'CESM2', 'CanESM5'],
             table_id='6hrLev',
             experiment_id=['amip', 'ssp585'],
             variable_id=['va', 'ua', 'ta', 'ps', 'hus']
)

cat = col.search(**query)

In [ ]:
# load data into dictionary
# keys are constructed as 'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

warnings.filterwarnings("ignore")

z_kwargs = {'consolidated': True, 'decode_times':True}

with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    dset_dict = cat.to_dataset_dict(zarr_kwargs=z_kwargs, preprocess=combined_preprocessing)

dset_dict.keys()

In [ ]:
dsw = dset_dict['CFMIP.NOAA-GFDL.GFDL-CM4.amip-p4K.Eday.gr2']
dsh = dset_dict['CMIP.NOAA-GFDL.GFDL-CM4.amip.Eday.gr2']

In [ ]:
dsw

In [ ]:
dsw.sel(x=301.25, y=-19).isel(time=9765).va.plot()